# Linkování C kódu a C knihoven s Pythonem

V této části si ukážeme, jak lze v Pythonu používat C knihovny a kód. 

- Využijeme knihovnu `ctypes` případně `cffi` (pro načítání knihoven) 
- Nativní Pythonovské rozhraní pro C, případně `cython`

## Nejprve C kód
Máme jednoduchý C kód, který provádí násobení matice a vektoru. 

Krátké opakování C:
- `*` před proměnnou znamená, že se jedná o ukazatel
- pole se v C zapisuje jako ukazatel na první prvek, sami o sobě neobsahují informaci o délce a tvaru
- `void` znamená, že funkce nic nevrací


In [ ]:
%%writefile matrix_vector_multiply.c

void matrix_vector_multiply(float *matrix, float *vector, float *result, int rows, int cols) {
    for (int i = 0; i < rows; i++) {
        result[i] = 0;
        for (int j = 0; j < cols; j++) {
            result[i] += matrix[i * cols + j] * vector[j];
        }
    }
}


#### Kompilejme C kód
Pro uživatele linuxu je tento krok velmi jednoduchý, neboť všechny potřebné komponenty již máte v systému.

Pokud se Vám pod Windows podaří zkompilovat C kód (ať už Microsoft Visual Studio C++ compiler (MSVC) nebo MinGW-w64) bude vašim výstupem `.dll` knihovna, tak pak pozor při načítání. V tomto případě bude potřeba použít `WinDLL` místo `CDLL`.

In [ ]:
!gcc -shared -fPIC -o libmatrix_vector_multiply.so matrix_vector_multiply.c